In [1]:
%load_ext jupyter_black
import numpy as np
import torch
from torch_pricing import asian_option, vmap_asian_option
from scipy.stats.qmc import Halton
from collections import deque
import matplotlib.pyplot as plt
import tqdm

seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


sampler = Halton(d=6, scramble=True, seed=seed)

parameter_space = dict(
    spot=(30, 70),
    path_integral=(25, 150),
    ttm=(0.2, 1),
    t=(0, 0.8),
    vol=(0.1, 0.5),
    r=(0, 0.1),
)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

The black scholes model for the spot price under risk neutral measure is:
$$dS_t = rS_t\,dt + \sigma S_t\,dW(t)$$
Which is equivalent to:
$$d\log(S_t) = \left(r - \frac{\sigma^2}{2}\right)\,dt + \sigma\,dW(t)$$
The corresponding Euler-Maruyama scheme for a discrete grid $\{t_k = \frac{kT}{n},\; k\in\{0,...,n\}\}$ is:
$$\log(S_{t_{k+1}})  = \log(S_{t_k}) + \left(r - \frac{\sigma^2}{2}\right)\frac{T}{n} + \sigma \sqrt\frac{T}{n} Z_{k+1}$$
Where $Z$ are i.i.d standard normal variables.

We want to compute the following price :
$$F(t, T, S_t, I_t, r, \sigma) = e^{-r(T-t)}\mathbb E\left[\left(S_T - I_T\right)^+\Big|\,S_t, \,I_t\right]$$
$$\text{Where :}\quad I_t = \frac{1}{t}\int_{0}^{t} S_u\,du$$

We want to learn $F(t, T, S_t, I_t, r, \sigma)$ for a predefined grid of parameters via an MLP, this translates to finding:
$$\theta^* \in \argmin_{\theta\in\Theta}\mathbb E_{x\sim D}\left[(F(x) - T_\theta(x))^2\right]$$
Where $x = (t, T, S_t, I_t, r, \sigma)$ and $D$ is a prior distribution over the parameter space. Since we are attempting to learn an expectation, we can rewrite the problem as:
$$\theta^* \in \argmin_{\theta\in\Theta}\mathbb E_{x\sim D}\left[\mathbb E_{(S_T, I_T)}\left[ (S_T - I_T)^+ - T_\theta(x))^2 \Big | x\right]\right]$$

We can thus train our network on payoffs and test it agaisnt MC estimations of the price.

In [3]:
# Numerical test
NTPB = 1024
NB = 1024
# n = NB * NTPB
n = 1000
T = 1.0
S_0 = 50.0
sigma = 0.2
r = 0.1
N = 100
t = 0.2
I = 60.0

asian_option(int(1e6), N, S_0, I, t, torch.sqrt(torch.tensor(T / N)), r, sigma).mean()

tensor(3.1666, device='cuda:0')

### Generating training data

In [4]:
torch.cuda.empty_cache()

In [5]:
# Sampling parameters from a grid
n = 10000
N = 100
nb_samples = int(1e6)
sample_params = sampler.random(n=nb_samples)
sample_params = np.array(
    [
        parameter_space["spot"][1] - parameter_space["spot"][0],
        parameter_space["path_integral"][1] - parameter_space["path_integral"][0],
        parameter_space["t"][1] - parameter_space["t"][0],
        parameter_space["ttm"][1] - parameter_space["ttm"][0],
        parameter_space["r"][1] - parameter_space["r"][0],
        parameter_space["vol"][1] - parameter_space["vol"][0],
    ]
) * sample_params + np.array(
    [
        parameter_space["spot"][0],
        parameter_space["path_integral"][0],
        parameter_space["t"][0],
        parameter_space["ttm"][0],
        parameter_space["r"][0],
        parameter_space["vol"][0],
    ]
)

sample_params[:, 1] = (sample_params[:, 3] < 0.05) * sample_params[:, 0] + (
    sample_params[:, 3] >= 0.05
) * sample_params[:, 0] * np.random.uniform(low=0.5, high=2, size=len(sample_params))

In [6]:
# Seperate parameters into batches
sample_params = torch.tensor(sample_params).to(device)
data_loader = torch.utils.data.DataLoader(sample_params, batch_size=512)

In [7]:
# Compute n payoffs per sample param
results = deque()
with tqdm.tqdm(
    total=len(data_loader), desc="Generating training data", position=0, leave=True
) as progress_bar:
    for sample in data_loader:
        sample_payoffs = vmap_asian_option(
            n,
            N,
            sample[:, 0],
            sample[:, 1],
            sample[:, 2],
            torch.sqrt(sample[:, 3] / N),
            sample[:, 4],
            sample[:, 5],
        ).mean(axis=1)
        sample_payoffs = sample_payoffs.to("cpu")
        results.append(sample_payoffs)
        progress_bar.update(1)

Generating training data: 100%|██████████| 1954/1954 [06:05<00:00,  5.34it/s]


In [8]:
# Saving data
X_train = sample_params.to("cpu").numpy()
with open("../data/X_train.npy", "wb") as f:
    np.save(f, X_train)

In [9]:
with open("../data/Y_train_averaged_10kpaths.npy", "wb") as f:
    np.save(f, torch.cat(list(results), dim=0).numpy(), allow_pickle=True)

### Generating validation data

In [10]:
torch.cuda.empty_cache()

In [11]:
n = NB * NTPB
N = 100
nb_samples = int(1e4)
sample_params = sampler.random(n=nb_samples)
sample_params = np.array(
    [
        parameter_space["spot"][1] - parameter_space["spot"][0],
        parameter_space["path_integral"][1] - parameter_space["path_integral"][0],
        parameter_space["t"][1] - parameter_space["t"][0],
        parameter_space["ttm"][1] - parameter_space["ttm"][0],
        parameter_space["r"][1] - parameter_space["r"][0],
        parameter_space["vol"][1] - parameter_space["vol"][0],
    ]
) * sample_params + np.array(
    [
        parameter_space["spot"][0],
        parameter_space["path_integral"][0],
        parameter_space["t"][0],
        parameter_space["ttm"][0],
        parameter_space["r"][0],
        parameter_space["vol"][0],
    ]
)

sample_params[:, 1] = (sample_params[:, 3] < 0.05) * sample_params[:, 0] + (
    sample_params[:, 3] >= 0.05
) * sample_params[:, 0] * np.random.uniform(low=0.5, high=2, size=len(sample_params))

In [12]:
# Seperate parameters into batches
sample_params = torch.tensor(sample_params).to(device)
data_loader = torch.utils.data.DataLoader(sample_params, batch_size=5)

In [13]:
# Compute n payoffs per sample param and retrieve mc estimator of price
results = deque()
with tqdm.tqdm(
    total=len(data_loader), desc="Generating validation data", position=0, leave=True
) as progress_bar:
    for sample in data_loader:
        sample_payoffs = vmap_asian_option(
            n,
            N,
            sample[:, 0],
            sample[:, 1],
            sample[:, 2],
            torch.sqrt(sample[:, 3] / N),
            sample[:, 4],
            sample[:, 5],
        ).mean(axis=1)
        sample_payoffs = sample_payoffs.to("cpu")
        results.append(sample_payoffs)
        progress_bar.update(1)

Generating validation data: 100%|██████████| 2000/2000 [10:09<00:00,  3.28it/s]


In [14]:
# Saving data
X_valid = sample_params.to("cpu").numpy()
with open("../data/X_valid.npy", "wb") as f:
    np.save(f, X_valid)

In [15]:
with open("../data/Y_valid.npy", "wb") as f:
    np.save(f, torch.cat(list(results), dim=0).numpy())